In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as sco
import matplotlib.pyplot as plt
import random

In [ ]:
#计算最优化权重的函数

#风险平价优化
def obj_fun_1(weight,cov_mat):
    arr_1=np.dot(cov_mat,weight)
    arr_2=weight*arr_1

    s = 0
    for x_1 in arr_2:
        for x_2 in arr_2:
            s+=(x_1-x_2)**2
    return s
    
       
def opt_1(cov_mat):
    num=cov_mat.shape[0]
    cons=({"type":"eq","fun":lambda x:np.sum(x)-1})
    bound=tuple((0,1) for x in range(num))
    #options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
    opt_weight=sco.minimize(obj_fun_1,num*[1/num],args=cov_mat,method="SLSQP",bounds=bound,constraints=cons)["x"]
    return opt_weight 

#最小化方差优化
def obj_fun_2(weight,cov_mat):
    variance=np.dot(np.dot(weight.T,cov_mat),weight)
    return variance

def opt_2(cov_mat):
    num=cov_mat.shape[0]
    cons=({"type":"eq","fun":lambda x:np.sum(x)-1})
    bound=tuple((0,1) for x in range(num))
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
    opt_weight=sco.minimize(obj_fun_2,num*[1/num],args=cov_mat,method="SLSQP",bounds=bound,constraints=cons,options=options)["x"]
    return opt_weight

#均值方差优化
def obj_fun_4(weight,ret_arr,cov_mat):
    variance=np.dot(np.dot(weight.T,cov_mat),weight)
    ret = np.sum(ret_arr*weight)
    return 0.5*variance-ret

def opt_4(ret_arr,cov_mat):
    num=cov_mat.shape[0]
    cons=({"type":"eq","fun":lambda x:np.sum(x)-1})
    bound=tuple((0,1) for x in range(num))
    #options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
    opt_weight=sco.minimize(obj_fun_4,num*[1/num],args=(ret_arr,cov_mat),method="SLSQP",bounds=bound,constraints=cons)["x"]
    return opt_weight

In [ ]:
#读入各行业日频收益率数据并进行形式上的处理
data_df=pd.read_csv("indus_ret.csv")
data_df.sort_values(by=["trade_date","industry_code"],ascending=["True","True"],inplace=True)

data_df.reset_index(drop=True,inplace=True)

grouped=data_df.groupby("trade_date")
date_list=list(data_df["trade_date"].drop_duplicates())[:-39]
code_list=list(data_df["industry_code"].drop_duplicates())

ret_arr=np.empty(shape=(len(code_list),0))

for date in date_list:
    change_rate=np.array(grouped.get_group(date)["change_rate"])
    ret_arr=np.column_stack([ret_arr,change_rate])

indus_ret_df=pd.DataFrame(ret_arr.T)

for i in range(28):
    indus_ret_df[str(i)+"_In_change"]=np.array(np.log(1+indus_ret_df[i]/100))[::-1]
    indus_ret_df[str(i)+"_indus_7d_ret"]=(np.array(np.exp(indus_ret_df[str(i)+"_In_change"].rolling(7).apply(np.sum)))[::-1]-1)*100
    del indus_ret_df[str(i)+"_In_change"]

In [ ]:
#计算Newey_West调整后的协方差矩阵
def Newey_West(arr,half_time):
    num=arr.shape[1]
    
    lamd=pow(0.5,1/half_time)
    weight=np.power(lamd,np.arange(252))[::-1]
    weight=weight/np.sum(weight)
    
    def f1(index):
        i=int(index/num)
        j=int(index%num)
        f_1 = arr[:, i]
        f_2 = arr[:, j]
        return np.sum(weight*(f_1-np.sum(weight*f_1))*(f_2-np.sum(weight*f_2)))+2/3*np.sum(weight[1:] * (f_1[1:] - np.sum(weight[1:]*f_1[1:])) * (f_2[:-1] - np.sum(weight[1:]*f_2[:-1])))\
               +2/3*np.sum(weight[1:] * (f_1[:-1] - np.sum(weight[1:]*f_1[:-1])) * (f_2[1:] - np.sum(weight[1:]*f_2[1:])))+1/3*np.sum(weight[2:] * (f_1[2:] - np.sum(weight[2:]*f_1[2:])) * (f_2[:-2] - np.sum(weight[2:]*f_2[:-2])))\
               +1/3*np.sum(weight[2:] * (f_1[:-2] - np.sum(weight[2:]*f_1[:-2])) * (f_2[2:] - np.sum(weight[2:]*f_2[2:])))
    pf1=np.vectorize(f1,otypes=[float])
    F_NW=pf1(np.arange(num**2)).reshape(num,num)
    
    return F_NW

In [ ]:
#对Newey_West调整后的协方差矩阵进行特征值调整
def Eigen_adjust(cov_matrix):
    factor_num=cov_matrix.shape[0]
    eigvalue,eigvector=np.linalg.eig(cov_matrix)

    bias_vec=np.zeros(factor_num,dtype=float)
    
    #进行1000次模拟，算出平均偏差
    for j in range(1000):
        eigfactor_matrix=np.empty(shape=(0,252),dtype=float)
        for i in range(factor_num):
            vec=np.random.normal(0,np.sqrt(eigvalue[i]),252)
            eigfactor_matrix=np.row_stack([eigfactor_matrix,vec])

        factor_matrix=np.dot(eigvector,eigfactor_matrix)

        F_MC=np.cov(factor_matrix)

        eigvalue_mc,eigvector_mc=np.linalg.eig(F_MC)

        eigvalue_true=np.diag(np.dot(np.dot(eigvector_mc.transpose(),cov_matrix),eigvector_mc))

        bias=eigvalue_true/eigvalue_mc

        bias_vec+=bias/1000

    #对平均偏差进行旋转
    bias_vec=np.sqrt(bias_vec)
    bias_vec=1.2*(bias_vec-1)+1
    bias_vec=np.power(bias_vec,2)
    eigvalue_adjust=bias_vec*eigvalue
    #print(bias_vec)
    #print(eigvalue)
    #print(eigvalue_adjust)
    
    #计算出新的协方差矩阵
    new_cov_matrix=np.dot(np.dot(eigvector,np.diag(eigvalue_adjust)),eigvector.transpose())
    
    return new_cov_matrix

In [ ]:
#对特征值调整后的协方差矩阵进行偏误调整
def Bias_adjust(ret_df,date):
    #计算各个日期的权重（时间越近，权重越高）
    lamd = pow(0.5, 1 / 45)
    weight = np.power(lamd, np.arange(126))[::-1]
    weight = weight / np.sum(weight)
    
    
    f_vol_multi=0
    for i in range(126):
        #获得每日风险预测值
        _date_=date-i
        risk=np.sqrt(np.diag(Eigen_dict[_date_]))
        
        #计算每日的Bias统计量
        Bias_t=0
        for j in range(len(risk)):
            risk_fore=risk[j]
            Bias_t+=(ret_df.iloc[date-i,j]/risk_fore)**2
        Bias_t/=len(risk)
        
        f_vol_multi+=Bias_t*weight[i]

    #print(f_vol_multi)
    Bias_dict[date]=f_vol_multi*Eigen_dict[date]

In [ ]:
#对不同调整阶段的协方差矩阵进行风险平价优化
date_list=list(indus_ret_df.index)
weight_dict_1={}
weight_dict_2={}
weight_dict_3={}
weight_dict_4={}

ret_df=indus_ret_df
Raw_dict={}
NW_dict={}
Eigen_dict={}
for date in range(252,len(date_list)):
    print(date,end=" ")
    df=pd.DataFrame(ret_df.iloc[date-252:date,0:28])
    
    Raw_dict[date]=np.array(df.cov())
    
    cov_matrix=Newey_West(np.array(df),180)
    NW_dict[date]=cov_matrix
    cov_matrix=Eigen_adjust(cov_matrix)
    Eigen_dict[date]=cov_matrix
   
Bias_dict={}
for i in range(378,len(date_list),7):
    Bias_adjust(ret_df,i)

for date in range(378,len(date_list),7):
    print("optimize ",date)
    ret_arr=np.array(ret_df.iloc[date-7,28:56])/7
    cov_matrix_1=Raw_dict[date]
    weight_dict_1[date]=opt_1(cov_matrix_1)
    
    cov_matrix_2=NW_dict[date]
    weight_dict_2[date]=opt_1(cov_matrix_2)
    
    cov_matrix_3=Eigen_dict[date]
    weight_dict_3[date]=opt_1(cov_matrix_3)
    
    cov_matrix_4=Bias_dict[date]
    weight_dict_4[date]=opt_1(cov_matrix_4)


In [ ]:
#对不同调整阶段的协方差矩阵进行均值方差优化
date_list=list(indus_ret_df.index)
weight_dict_1={}
weight_dict_2={}
weight_dict_3={}
weight_dict_4={}

ret_df=indus_ret_df
Raw_dict={}
NW_dict={}
Eigen_dict={}
for date in range(252,len(date_list)):
    print(date,end=" ")
    df=pd.DataFrame(ret_df.iloc[date-252:date,0:28])
    
    Raw_dict[date]=np.array(df.cov())
    
    cov_matrix=Newey_West(np.array(df),180)
    NW_dict[date]=cov_matrix
    cov_matrix=Eigen_adjust(cov_matrix)
    Eigen_dict[date]=cov_matrix
   
Bias_dict={}
for i in range(378,len(date_list),7):
    Bias_adjust(ret_df,i)

for date in range(378,len(date_list),7):
    print("optimize ",date)
    ret_arr=np.array(ret_df.iloc[date-7,28:56])/7
    cov_matrix_1=Raw_dict[date]
    weight_dict_1[date]=opt_4(ret_arr,cov_matrix_1)
    
    cov_matrix_2=NW_dict[date]
    weight_dict_2[date]=opt_4(ret_arr,cov_matrix_2)
    
    cov_matrix_3=Eigen_dict[date]
    weight_dict_3[date]=opt_4(ret_arr,cov_matrix_3)
    
    cov_matrix_4=Bias_dict[date]
    weight_dict_4[date]=opt_4(ret_arr,cov_matrix_4)


In [ ]:
#组合优化结果展示图

zz500=pd.read_csv("zz500.csv")
zz500["In_change"]=np.array(np.log(1+zz500["change_rate"]/100))[::-1]
zz500["mkt_21d_ret"]=(np.array(np.exp(zz500["In_change"].rolling(7).apply(np.sum)))[::-1]-1)*100

mkt_ret_arr=np.array(zz500["mkt_21d_ret"])[504::7]
cum_mkt_arr=np.cumprod(1+mkt_ret_arr/100)
plt.plot(cum_mkt_arr,color="blue",label="bench_mark")

port_ret_ls1=[]
port_ret_ls2=[]
port_ret_ls3=[]
port_ret_ls4=[]
port_ret_ls5=[]

for date in range(504,len(date_list),7):
    ret_arr=(ret_df.iloc[date,28:56])
    
    port_ret1=np.sum(weight_dict_1[date]*ret_arr)
    port_ret2=np.sum(weight_dict_2[date]*ret_arr)
    port_ret3=np.sum(weight_dict_3[date]*ret_arr)
    port_ret4=np.sum(weight_dict_4[date]*ret_arr)
    port_ret5=np.mean(ret_arr)
    
    port_ret_ls1.append(port_ret1)
    port_ret_ls2.append(port_ret2)
    port_ret_ls3.append(port_ret3)
    port_ret_ls4.append(port_ret4)
    port_ret_ls5.append(port_ret5)
    
port_ret_arr1=np.array(port_ret_ls1)
cum_arr1=np.cumprod(1+port_ret_arr1/100)
port_ret_arr2=np.array(port_ret_ls2)
cum_arr2=np.cumprod(1+port_ret_arr2/100)
port_ret_arr3=np.array(port_ret_ls3)
cum_arr3=np.cumprod(1+port_ret_arr3/100)
port_ret_arr4=np.array(port_ret_ls4)
cum_arr4=np.cumprod(1+port_ret_arr4/100)
port_ret_arr5=np.array(port_ret_ls5)
cum_arr5=np.cumprod(1+port_ret_arr5/100)

#绝对收益图
plt.plot(cum_arr1,color="red",label="Raw")
plt.plot(cum_arr2,color="green",label="Newey_West")
plt.plot(cum_arr3,color="black",label="Eigen_adjust")
plt.plot(cum_arr4,color="brown",label="Bias_adjust")
plt.plot(cum_arr5,color="purple",label="equal_weight")
plt.legend()
plt.savefig("plot_abso.jpg",dpi=500)
plt.show()

#相对收益图
relative_arr1=cum_arr1/cum_mkt_arr
relative_arr2=cum_arr2/cum_mkt_arr
relative_arr3=cum_arr3/cum_mkt_arr
relative_arr4=cum_arr4/cum_mkt_arr
relative_arr5=cum_arr5/cum_mkt_arr
plt.plot(relative_arr1,color="red",label="Raw")
plt.plot(relative_arr2,color="green",label="Newey_West")
plt.plot(relative_arr3,color="black",label="Eigen_adjust")
plt.plot(relative_arr4,color="brown",label="Bias_adjust")
plt.plot(relative_arr5,color="purple",label="equal_weight")
plt.legend()
plt.savefig("plot_rela.jpg",dpi=500)
plt.show()

In [ ]:
#组合优化指标对比表格

def indicator_calcu(arr):
    vola=np.std(arr)
    cum_arr=np.cumprod(1+arr/100)
    max_retract=(np.max(cum_arr)-np.min(cum_arr[np.argmax(cum_arr):]))/np.max(cum_arr)
    sharpe=np.mean(arr)/np.std(arr)
    
    year_ret=(cum_arr[-1]-1)*100*36/len(cum_arr)
    
    return year_ret,vola,max_retract,sharpe

df=pd.DataFrame(index=["Raw","Newey_West","Eigen_adjust","Bias_adjust","equal_weight"],columns=["yearly_ret","vloa","max_retract","sharpe"])
df.iloc[0,:]=list(indicator_calcu(port_ret_arr1[:-1]))
df.iloc[1,:]=list(indicator_calcu(port_ret_arr2[:-1]))
df.iloc[2,:]=list(indicator_calcu(port_ret_arr3[:-1]))
df.iloc[3,:]=list(indicator_calcu(port_ret_arr4[:-1]))
df.iloc[4,:]=list(indicator_calcu(port_ret_arr5[:-1]))
df

In [ ]:
#读取股票收益数据并随机抽取500只股票进行计算
alpha_df=pd.read_csv("equal_weight ZScore_1.csv")

code_list=list(alpha_df["code"].drop_duplicates())
alpha_df.set_index("code",inplace=True)
drop_code=random.sample(code_list,1635)
alpha_df.drop(drop_code,inplace=True)
alpha_df.reset_index()

alpha_df.sort_values(by=["datetime","code"],ascending=[True,True],inplace=True)
ret_df=alpha_df.pivot_table("change",index="datetime",columns="code")
mkt_value_df=alpha_df.pivot_table("market_value",index="datetime",columns="code")
stock_num=len(ret_df.columns)

In [ ]:
def zjh(arr):
    arr_t=arr.transpose()

    #计算重叠矩阵
    overlap_matrix=np.dot(arr_t,arr)
    overlap_matrix=(overlap_matrix.T+overlap_matrix)/2
    #计算特征值、特征向量
    eigvalue,eigvector=np.linalg.eig(overlap_matrix)

    #处理对角矩阵
    eigvalue=eigvalue**(-0.5)
    DiagMat=np.diag(eigvalue)

    #计算最终正交矩阵
    tmp=np.dot(eigvector,DiagMat)
    TSMat=np.dot(tmp,eigvector.transpose())
    OrthMat=np.dot(arr,TSMat)
    
    return OrthMat

In [ ]:
##计算Newey_West调整后的特质性风险
def spec_NW(arr,half_time):
    #计算各个日期的权重（时间越近，权重越高）
    lamd=pow(0.5,1/half_time)
    weight=np.power(lamd,np.arange(252))[::-1]
    weight=weight/np.sum(weight)
    
    #计算股票特质性收益的方差
    stock_num=arr.shape[1]
    def f1(i):
        f_1 = arr[:,i]
        return np.sum(weight * (f_1 - np.sum(weight*f_1)) ** 2)
    pf1=np.vectorize(f1,otypes=[float])
    F_raw=pf1(np.arange(stock_num))

    def f2(i):
        f_1 = arr[:,i]
        return np.sum(weight[1:] * (f_1[:-1] - np.sum(weight[1:]*f_1[:-1])) * (f_1[1:] - np.sum(weight[1:]*f_1[1:])))
    pf2=np.vectorize(f2,otypes=[float])
    C_lag_1=pf2(np.arange(stock_num))

    def f3(i):
        f_1 = arr[:,i]
        return np.sum(weight[2:] * (f_1[:-2] -  np.sum(weight[2:]*f_1[:-2])) * (f_1[2:] - np.sum(weight[2:]*f_1[2:])))
    pf3=np.vectorize(f3,otypes=[float])
    C_lag_2=pf3(np.arange(stock_num))

    F_NW=F_raw+4/3*C_lag_1+2/3*C_lag_2

    return F_NW

In [ ]:
#对Newey_West调整后的特质性风险进行贝叶斯压缩调整
def Bayes_shrinkage(F_NW,mkt_value_df):
    mktv=mkt_value_df.iloc[date,:]
    d={"risk":np.sqrt(F_NW),"market_value":mktv}
    specrisk_df=pd.DataFrame(data=d)
    
    #按市值排名进行分组
    mkt_arr=np.array(specrisk_df["market_value"])
    mkt_rank = mkt_arr.argsort().argsort()
    specrisk_df["mkt_rank"]=mkt_rank
    mkt_tag = pd.cut(specrisk_df.mkt_rank, 10, labels=False)
    specrisk_df["tag"] = mkt_tag
    
    #计算出各组风险的平均值和标准差
    avg = specrisk_df[["risk"]].groupby(specrisk_df["tag"]).agg([np.mean])
    std = specrisk_df[["risk"]].groupby(specrisk_df["tag"]).agg([np.std])
    specrisk_df["avg"]=specrisk_df["tag"].apply(lambda x:avg.iloc[x,0])
    specrisk_df["std"] = specrisk_df["tag"].apply(lambda x: std.iloc[x, 0])
    
    #进行贝叶斯压缩估计（系数q设为1）
    q=1
    specrisk_df["v"]=q*np.abs(specrisk_df["risk"]-specrisk_df["avg"])/(np.abs(q*(specrisk_df["risk"]-specrisk_df["avg"]))+specrisk_df["std"])
    specrisk_df["BS_risk"]=specrisk_df["v"]*specrisk_df["avg"]+(1-specrisk_df["v"])*specrisk_df["risk"]
    return np.power(np.array(specrisk_df["BS_risk"]),2)

In [ ]:
#对贝叶斯压缩调整后的股票特质性收益进行偏误调整
def spec_Bias_adjust(ret_df,date):
    #计算各个日期的权重（时间越近，权重越高）
    lamd = pow(0.5, 1 / 45)
    weight = np.power(lamd, np.arange(126))[::-1]
    weight = weight / np.sum(weight)
    
    
    f_vol_multi=0
    for i in range(126):
        #获得每日风险预测值
        _date_=date-i
        risk=np.sqrt(BS_dict[date])
        
        #计算每日的Bias统计量
        Bias_t=0
        j=np.arange(len(risk))
        Bias_t=np.sum(np.power(ret_df.iloc[date-i,j]/risk,2))/len(risk)
        
        f_vol_multi+=Bias_t*weight[i]

    print(f_vol_multi)
    Bias_dict[date]=f_vol_multi*BS_dict[date]

In [ ]:
#对不同调整阶段的特质性风险进行风险平价优化
date_list=list(ret_df.index)
weight_dict_1={}
weight_dict_2={}
weight_dict_3={}
weight_dict_4={}

Raw_dict={}
NW_dict={}
BS_dict={}

ret_df=pd.DataFrame(zjh(np.array(ret_df)))

for date in range(252,len(date_list)):
    print(date,end=" ")
    df=pd.DataFrame(ret_df.iloc[date-252:date,0:stock_num])
    
    Raw_dict[date]=np.diag(np.array(df.cov()))
    
    spec_risk=spec_NW(np.array(df),180)
    NW_dict[date]=spec_risk
    spec_risk=Bayes_shrinkage(spec_risk,mkt_value_df)
    BS_dict[date]=spec_risk

    
    
for code in ret_df.columns:
    ret_df[code+"_In_change"]=np.array(np.log(1+ret_df[code]/100))[::-1]
    ret_df[code+"_7d_ret"]=(np.array(np.exp(ret_df[code+"_In_change"].rolling(7).apply(np.sum)))[::-1]-1)*100
    del ret_df[code+"_In_change"]
    
    
Bias_dict={}
for i in range(378,len(date_list),7):
    spec_Bias_adjust(ret_df,i)

for date in range(378,len(date_list),7):
    print("optimize ",date)
    spec_risk_1=Raw_dict[date]
    weight_dict_1[date]=spec_risk_1**(-0.5)/np.sum(spec_risk_1**(-0.5))
    
    spec_risk_2=NW_dict[date]
    weight_dict_2[date]=spec_risk_2**(-0.5)/np.sum(spec_risk_2**(-0.5))
    
    spec_risk_3=BS_dict[date]
    weight_dict_3[date]=spec_risk_3**(-0.5)/np.sum(spec_risk_3**(-0.5))
    
    spec_risk_4=Bias_dict[date]
    weight_dict_4[date]=spec_risk_4**(-0.5)/np.sum(spec_risk_4**(-0.5))

In [ ]:
#组合优化结果展示图

zz500=pd.read_csv("zz500.csv")
zz500["In_change"]=np.array(np.log(1+zz500["change_rate"]/100))[::-1]
zz500["mkt_21d_ret"]=(np.array(np.exp(zz500["In_change"].rolling(7).apply(np.sum)))[::-1]-1)*100

mkt_ret_arr=np.array(zz500["mkt_21d_ret"])[504::7]
cum_mkt_arr=np.cumprod(1+mkt_ret_arr/100)
plt.plot(cum_mkt_arr,color="blue",label="bench_mark")

port_ret_ls1=[]
port_ret_ls2=[]
port_ret_ls3=[]
port_ret_ls4=[]
port_ret_ls5=[]

for date in range(504,len(date_list),7):
    ret_arr=(ret_df.iloc[date,stock_num:2*stock_num])
    
    port_ret1=np.sum(weight_dict_1[date]*ret_arr)
    port_ret2=np.sum(weight_dict_2[date]*ret_arr)
    port_ret3=np.sum(weight_dict_3[date]*ret_arr)
    port_ret4=np.sum(weight_dict_4[date]*ret_arr)
    port_ret5=np.mean(ret_arr)
    
    port_ret_ls1.append(port_ret1)
    port_ret_ls2.append(port_ret2)
    port_ret_ls3.append(port_ret3)
    port_ret_ls4.append(port_ret4)
    port_ret_ls5.append(port_ret5)
    
port_ret_arr1=np.array(port_ret_ls1)
cum_arr1=np.cumprod(1+port_ret_arr1/100)
port_ret_arr2=np.array(port_ret_ls2)
cum_arr2=np.cumprod(1+port_ret_arr2/100)
port_ret_arr3=np.array(port_ret_ls3)
cum_arr3=np.cumprod(1+port_ret_arr3/100)
port_ret_arr4=np.array(port_ret_ls4)
cum_arr4=np.cumprod(1+port_ret_arr4/100)
port_ret_arr5=np.array(port_ret_ls5)
cum_arr5=np.cumprod(1+port_ret_arr5/100)

#绝对收益图
plt.plot(cum_arr1,color="red",label="Raw")
plt.plot(cum_arr2,color="green",label="Newey_West")
plt.plot(cum_arr3,color="black",label="Bayes_Shrinkage")
plt.plot(cum_arr4,color="brown",label="Bias_adjust")
plt.plot(cum_arr5,color="purple",label="equal_weight")
plt.legend()
plt.savefig("plot_abso.jpg",dpi=500)
plt.show()

#相对收益图
relative_arr1=cum_arr1/cum_mkt_arr
relative_arr2=cum_arr2/cum_mkt_arr
relative_arr3=cum_arr3/cum_mkt_arr
relative_arr4=cum_arr4/cum_mkt_arr
relative_arr5=cum_arr5/cum_mkt_arr
plt.plot(relative_arr1,color="red",label="Raw")
plt.plot(relative_arr2,color="green",label="Newey_West")
plt.plot(relative_arr3,color="black",label="Bayes_Shrinkage")
plt.plot(relative_arr4,color="brown",label="Bias_adjust")
plt.plot(relative_arr5,color="purple",label="equal_weight")
plt.legend()
plt.savefig("plot_rela.jpg",dpi=500)
plt.show()

In [ ]:
#组合优化指标对比表格

def indicator_calcu(arr):
    vola=np.std(arr)
    cum_arr=np.cumprod(1+arr/100)
    max_retract=(np.max(cum_arr)-np.min(cum_arr[np.argmax(cum_arr):]))/np.max(cum_arr)
    sharpe=np.mean(arr)/np.std(arr)
    
    year_ret=(cum_arr[-1]-1)*100*36/len(cum_arr)
    
    return year_ret,vola,max_retract,sharpe

df=pd.DataFrame(index=["Raw","Newey_West","Bayes_Shrinkage","Bias_adjust","equal_weight"],columns=["yearly_ret","vloa","max_retract","sharpe"])
df.iloc[0,:]=list(indicator_calcu(port_ret_arr1[:-1]))
df.iloc[1,:]=list(indicator_calcu(port_ret_arr2[:-1]))
df.iloc[2,:]=list(indicator_calcu(port_ret_arr3[:-1]))
df.iloc[3,:]=list(indicator_calcu(port_ret_arr4[:-1]))
df.iloc[4,:]=list(indicator_calcu(port_ret_arr5[:-1]))
df